In [72]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.cluster import MeanShift
import matplotlib.pyplot as plt
%matplotlib inline

In [73]:
data = pd.read_csv('checkins.dat',sep='|').replace(r'^\s+$', np.nan, regex=True).dropna()
print data.shape
data.head()

C:\Users\Ivan\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(396634, 6)


,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18


In [74]:
dataset = pd.DataFrame()
dataset['latitude'] = data[data.columns[3]].apply(lambda x: float(x))
dataset['longitude'] = data[data.columns[4]].apply(lambda x: float(x))
print dataset.shape
dataset.head()

(396634, 2)


,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037


In [76]:
subset = dataset[:100000]
print subset.shape
subset.head()

(100000, 2)


,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037


In [91]:
subset[subset.columns[0]]

2         38.895112
4         33.800745
8         45.523452
10        40.764462
11        33.448377
12        32.221743
13        40.650000
14        33.448377
16        33.414768
19        42.358431
20        33.448377
21        33.414768
26        33.448377
27        33.248664
28        32.840677
30        34.052234
35        33.414768
36        33.448377
39        34.483901
41        34.048928
42        32.715329
43        33.448377
47        38.895112
48        33.745851
50        34.096111
51        34.052234
53        37.629349
54         4.598056
55        33.804167
56        40.735657
            ...    
233729    32.678109
233731    34.019454
233732    37.339386
233733    40.650000
233736    43.009703
233737    34.746481
233738    41.763711
233739    44.310000
233740    35.221997
233742    40.740389
233743    33.641132
233744    40.715972
233745    41.033986
233747    27.937801
233750    36.087708
233752    26.203407
233754    40.740389
233756    40.768433
233759    42.866680


In [96]:
%%time
model = MeanShift(bandwidth=0.1)
preds = model.fit(subset)

Wall time: 16min 3s


In [97]:
preds.cluster_centers_.shape

(3229L, 2L)

In [122]:
preds.cluster_centers_

array([ 40.7177164 , -73.99183542])

In [110]:
offices = np.array([[33.751277, -118.188740],[25.867736, -80.324116],[51.503016, -0.075479],[52.378894, 4.885084],
                      [39.366487, 117.036146],[-33.868457, 151.205134]]) 

In [121]:
offices

array([[  3.37512770e+01,  -1.18188740e+02],
       [  2.58677360e+01,  -8.03241160e+01],
       [  5.15030160e+01,  -7.54790000e-02],
       [  5.23788940e+01,   4.88508400e+00],
       [  3.93664870e+01,   1.17036146e+02],
       [ -3.38684570e+01,   1.51205134e+02]])

In [124]:
np.linalg.norm(offices[0] - preds.cluster_centers_[0])

44.742570917848042

In [138]:
distances = []
for center in preds.cluster_centers_:
    min_dist = 10e12
    for office in offices:
        dist = np.linalg.norm(center - office)
        if (dist < min_dist):
            min_dist = dist
            latitude = center[0]
            longtitude = center[1]
    distances.append([min_dist, latitude, longtitude])

In [141]:
def sort_fun(x):
    return x[0]

In [142]:
distances.sort(key = sort_fun)

In [143]:
distances

[[0.007834758163107856, -33.860630428571433, 151.20477592857145],
 [0.0093533161859922255, 52.37296399032261, 4.8923172225806466],
 [0.022674066158385495, 25.8456722642857, -80.318890596428574],
 [0.050058294822787869, 51.502991260887086, -0.12553728870967767],
 [0.070847732427199731, 33.809877955263097, -118.14892380690813],
 [0.13410903336184654, 25.785812419967499, -80.217938036825402],
 [0.15410282980602005, -34.001906149999996, 151.12806904999999],
 [0.1674059642503429, 25.705349721052592, -80.28342873815798],
 [0.18887596060185083, 26.010098249285683, -80.199990585714318],
 [0.19222726571703552, -33.952262900000001, 151.03213719999999],
 [0.19577945647763628, 33.888325342758598, -118.04892817172427],
 [0.21181053682436798, 33.872986011570177, -118.36209114655645],
 [0.22223329073179071, 33.97257482142858, -118.16837066666663],
 [0.24065314956287862, 51.426763288888885, -0.30373206666666669],
 [0.25150208676235281, 52.388500999999991, 4.6337654666666657],
 [0.26892862728711103, 51

In [147]:
answer = [distances[0][1], distances[0][2]]

In [151]:
answer[0]

-33.860630428571433

In [152]:
def write_answer(answer):
    with open("clustering_answer.txt", "w") as fout:
        fout.write(str(answer[0]) + ' ' + str(answer[1]))

In [153]:
write_answer(answer)